# Imports

In [ ]:
import matplotlib.pyplot as plt
from skimage import data, filters
from skimage.color import rgb2gray
from skimage.io import imread
import os
import numpy as np

# Binary Thresaholding

In [8]:
gtmasks_dirs = list(filter(lambda dname: dname.endswith('gtmasks'),os.listdir('Img')))
gtmasks_dirs = sorted(gtmasks_dirs, key=lambda dname: int(dname.split('_')[3]))
gtmasks_dict = {dname: os.listdir(f'Img/{dname}') for dname in gtmasks_dirs}
gtmasks_dict.keys()

dict_keys(['wire_images_video_1_gtmasks', 'wire_images_video_2_gtmasks', 'wire_images_video_3_gtmasks', 'wire_images_video_4_gtmasks', 'wire_images_video_5_gtmasks', 'wire_images_video_6_gtmasks', 'wire_images_video_7_gtmasks', 'wire_images_video_8_gtmasks'])

In [13]:
image_dirs = list(filter(lambda dname: dname[-1].isdigit(), os.listdir('Img')))
image_dirs = sorted(image_dirs, key=lambda dname: int(dname[-1]))
images_dict = {dname: os.listdir(f'Img/{dname}') for dname in image_dirs}
images_dict

{'wire_images_video_1': ['wire1_ultrasound_watertank.png',
  'wire2_ultrasound_watertank.png',
  'wire3_ultrasound_watertank.png',
  'wire4_ultrasound_watertank.png',
  'wire5_ultrasound_watertank.png',
  'wire6_ultrasound_watertank.png'],
 'wire_images_video_2': ['wire10_ultrasound_watertank.png',
  'wire11_ultrasound_watertank.png',
  'wire12_ultrasound_watertank.png',
  'wire13_ultrasound_watertank.png',
  'wire14_ultrasound_watertank.png',
  'wire15_ultrasound_watertank.png',
  'wire16_ultrasound_watertank.png',
  'wire17_ultrasound_watertank.png',
  'wire18_ultrasound_watertank.png',
  'wire19_ultrasound_watertank.png',
  'wire20_ultrasound_watertank.png',
  'wire21_ultrasound_watertank.png',
  'wire22_ultrasound_watertank.png',
  'wire23_ultrasound_watertank.png',
  'wire7_ultrasound_watertank.png'],
 'wire_images_video_3': ['wire24_ultrasound_watertank.png',
  'wire25_ultrasound_watertank.png',
  'wire26_ultrasound_watertank.png',
  'wire27_ultrasound_watertank.png',
  'wire28_u

## Thresholding Functions

In [83]:
def huang_threshold(image):
    hist, bin_edges = np.histogram(image.flatten(), bins=256, range=(0, 255))
    hist_norm = hist / hist.sum()  # Normalize the histogram
    
    # Cumulative sum and cumulative mean
    cum_sum = np.cumsum(hist_norm)
    cum_mean = np.cumsum(hist_norm * np.arange(256))
    
    # Overall mean
    mean_total = cum_mean[-1]
    
    entropy_values = []
    for t in range(1, 255):
        p1 = cum_sum[t]
        p2 = 1 - p1
        mean1 = cum_mean[t] / p1 if p1 != 0 else 0
        mean2 = (mean_total - cum_mean[t]) / p2 if p2 != 0 else 0
        
        # Entropy calculation
        entropy = -p1 * np.log(p1 + 1e-6) - p2 * np.log(p2 + 1e-6)
        entropy_values.append(entropy)
    
    # Find the threshold that minimizes the entropy
    threshold = np.argmin(entropy_values)
    return threshold


def max_entropy_threshold(image):
    hist, bin_edges = np.histogram(image.flatten(), bins=256, range=(0, 255))
    hist_norm = hist / hist.sum()  # Normalize the histogram
    
    # Cumulative sum and cumulative mean
    cum_sum = np.cumsum(hist_norm)
    cum_mean = np.cumsum(hist_norm * np.arange(256))
    
    # Overall mean
    mean_total = cum_mean[-1]
    
    entropy_values = []
    for t in range(1, 255):
        p1 = cum_sum[t]
        p2 = 1 - p1
        mean1 = cum_mean[t] / p1 if p1 != 0 else 0
        mean2 = (mean_total - cum_mean[t]) / p2 if p2 != 0 else 0
        
        # Entropy for foreground and background
        entropy1 = -p1 * np.log(p1 + 1e-6)
        entropy2 = -p2 * np.log(p2 + 1e-6)
        total_entropy = entropy1 + entropy2
        entropy_values.append(total_entropy)
    
    # Find the threshold that maximizes the entropy
    threshold = np.argmax(entropy_values)
    return threshold


def percentile_threshold(image, percentile):
    hist, bin_edges = np.histogram(image.flatten(), bins=256, range=(0, 255))
    hist_norm = hist / hist.sum()
    
    # Find the cumulative sum to get the percentile
    cum_sum = np.cumsum(hist_norm)
    threshold = np.searchsorted(cum_sum, percentile / 100.0)
    
    return threshold


def moments_threshold(image):
    hist, bin_edges = np.histogram(image.flatten(), bins=256, range=(0, 255))
    hist_norm = hist / hist.sum()  # Normalize the histogram
    
    total_mean = np.sum(np.arange(256) * hist_norm)
    best_threshold = 0
    min_moment_diff = float('inf')
    
    for t in range(1, 255):
        w0 = np.sum(hist_norm[:t])
        w1 = np.sum(hist_norm[t:])
        
        mean0 = np.sum(np.arange(t) * hist_norm[:t]) / w0 if w0 != 0 else 0
        mean1 = np.sum(np.arange(t, 256) * hist_norm[t:]) / w1 if w1 != 0 else 0
        
        moment_diff = (mean0 - total_mean) ** 2 + (mean1 - total_mean) ** 2
        if moment_diff < min_moment_diff:
            min_moment_diff = moment_diff
            best_threshold = t
    
    return best_threshold


def intermodes_threshold(image):
    hist, bin_edges = np.histogram(image.flatten(), bins=256, range=(0, 255))
    hist_norm = hist / hist.sum()  # Normalize the histogram
    
    # Iterate over all possible thresholds and calculate between-class variance
    max_variance = -1
    best_threshold = 0
    
    for t in range(1, 255):
        # Split the histogram into two parts: [0, t] and [t+1, 255]
        w0 = np.sum(hist_norm[:t])
        w1 = np.sum(hist_norm[t:])
        
        mean0 = np.sum(np.arange(t) * hist_norm[:t]) / w0 if w0 != 0 else 0
        mean1 = np.sum(np.arange(t, 256) * hist_norm[t:]) / w1 if w1 != 0 else 0
        
        # Calculate between-class variance
        variance = w0 * w1 * (mean0 - mean1) ** 2
        
        if variance > max_variance:
            max_variance = variance
            best_threshold = t
    
    return best_threshold


def renyi_entropy_threshold(image, alpha=2):
    hist, bin_edges = np.histogram(image.flatten(), bins=256, range=(0, 255))
    hist_norm = hist / hist.sum()  # Normalize the histogram
    
    # Calculate Renyi entropy
    entropy_values = []
    for t in range(1, 255):
        p1 = np.sum(hist_norm[:t])
        p2 = 1 - p1
        entropy = (1 / (1 - alpha)) * np.log(p1 ** alpha + p2 ** alpha + 1e-6)
        entropy_values.append(entropy)
    
    threshold = np.argmax(entropy_values)
    return threshold


def shanbhag_threshold(image):
    hist, bin_edges = np.histogram(image.flatten(), bins=256, range=(0, 255))
    hist_norm = hist / hist.sum()  # Normalize the histogram
    
    # Calculate cumulative sum and entropy
    cum_sum = np.cumsum(hist_norm)
    threshold_values = []
    
    for t in range(1, 255):
        p1 = cum_sum[t]
        p2 = 1 - p1
        entropy = -p1 * np.log(p1 + 1e-6) - p2 * np.log(p2 + 1e-6)
        threshold_values.append(entropy)
    
    threshold = np.argmax(threshold_values)
    return threshold

## General Functions

In [112]:
def get_image_path(folder_name, file_name):
    return os.path.join("Img", folder_name, file_name)

def display_image(image, title):
     # Display the filtered image
    plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

def filter_image_top(image):
   # Get the shape of the image
    height, width = image.shape    
    # Create a filter matrix with ones
    filter_matrix = np.ones((height, width))
    # Set rows y = 0 to y = 175 to zeros
    filter_matrix[0:176, :] = 0  # Note: Includes row 175
    # Multiply the filter matrix by the image
    filtered_image = image * filter_matrix
    return filtered_image

def is_8bit_image(image):
    """
    Check if the given image is an 8-bit grayscale or RGB image.

    Parameters:
        image (numpy.ndarray): The image to check.

    Returns:
        bool: True if the image is 8-bit, False otherwise.
    """
    if not isinstance(image, np.ndarray):
        raise ValueError("Input must be a numpy array.")
    
    # Check the data type and value range
    return image.dtype == np.uint8 and image.min() >= 0 and image.max() <= 255

def calculate_accuracy(ground_truth, prediction):
    """
    Calculate the accuracy of the predicted mask compared to the ground truth mask.

    Parameters:
    - ground_truth: 2D numpy array (8-bit) representing the ground truth mask.
    - prediction: 2D numpy array (8-bit) representing the predicted mask.

    Returns:
    - accuracy: A float value between 0 and 1 representing the accuracy.
    """
    # Ensure the inputs are numpy arrays
    ground_truth = np.array(ground_truth)
    prediction = np.array(prediction)
    
    # Check that both images have the same shape
    if ground_truth.shape != prediction.shape:
        raise ValueError("Ground truth and prediction must have the same shape")
    
    # Calculate the number of correct pixels
    correct_pixels = np.sum(ground_truth == prediction)
    
    # Calculate the total number of pixels
    total_pixels = ground_truth.size
    
    # Calculate the accuracy
    accuracy = correct_pixels / total_pixels
    return accuracy

def create_8_bit_image(image):
    return (image * 255).astype(np.uint8)


def apply_thresholding_method(image, method, threshold_value):
    """
    Apply the corresponding thresholding method to the image.
    This function supports various thresholding techniques based on the method name.
    """
    try:
        match method:
            case "Otsu":
                threshold = filters.threshold_otsu(image)
            case "Li":
                threshold = filters.threshold_li(image)
            case "Huang":
                threshold = huang_threshold(image)
            case "Intermodes":
                threshold = intermodes_threshold(image)
            case "IsoData":
                threshold = filters.threshold_isodata(image)
            case "MaxEntropy":
                threshold = max_entropy_threshold(image)
            case "Mean":
                threshold = filters.threshold_mean(image)
            case "Minimum":
                threshold = filters.threshold_minimum(image)
            case "Moments":
                threshold = moments_threshold(image)
            case "Percentile":
                threshold = percentile_threshold(image, percentile=threshold_value)
            case "RenyiEntropy":
                threshold = renyi_entropy_threshold(image)
            case "Shanbhag":
                threshold = shanbhag_threshold(image)
            case "Triangle":
                threshold = filters.threshold_triangle(image)
            case "Yen":
                threshold = filters.threshold_yen(image)
            case "Default":
                threshold = threshold_value
            case _:
                # If no specific method is matched, use the default threshold_value.
                raise ValueError(f"Unfimiliar method for binary thresholding: {method}")
        
    except AttributeError as e:
        raise ValueError(f"{method}")
    
    # Generate binary mask based on the threshold
    binary_mask = image > threshold
    return binary_mask


def print_scores(accuracies):
    sorted_accuracies = [(sum(accuracy_list)/len(accuracy_list), method, len(accuracy_list)) for method, accuracy_list in accuracies.items()]
    sorted_accuracies = sorted(sorted_accuracies, key=lambda x: x[0], reverse=True)
    # for method, accuracy_list in accuracies.items():
    #     size = len(accuracy_list)
    #     print(f"Method: {method} | Accuracy over {size} images: {sum(accuracy_list)/size}")
    for i, accuracy_tuple in enumerate(sorted_accuracies):
        print(f"{i+1}. Method: {accuracy_tuple[1]} | Accuracy over {accuracy_tuple[2]} images: {accuracy_tuple[0]}")

# Header

# Testing

In [12]:
# load dict for ground truth masks images
for dname, image_list in gtmasks_dict.items():
    for image in image_list:
        print(get_image_path(dname, image))
        break
    break

Img\wire_images_video_1_gtmasks\wire1_ultrasound_watertank_gtmask.png


In [92]:
# Thresholds from ImageJ
thresholds_imagej = {
    "Default": 45,
    "Huang": 6,
    "Intermodes": 113,
    "IsoData": 45,
    "Li": 10,
    "MaxEntropy": 140,
    "Mean": 5,
    "Minimum": 188,
    "Moments": 69,
    "Otsu": 45,
    "Percentile": 0,
    "RenyiEntropy": 6,
    "Shanbhag": 238,
    "Triangle": 2,
    "Yen": 3
}

In [114]:
accuracies = {method: [] for method in thresholds_imagej}
errors = []
lame_methods = set()
for image_dir, gtmask_dir in zip(images_dict, gtmasks_dict):
    for image_name, gtmask_name in zip(images_dict[image_dir], gtmasks_dict[gtmask_dir]):
        im_path = get_image_path(image_dir, image_name)
        gtm_path = get_image_path(gtmask_dir, gtmask_name)
        image = rgb2gray(imread(im_path))
        # display_image(image, im_path)
        # Filters the top of the image and converts it to an 8-bit image
        filtered_image = create_8_bit_image(filter_image_top(image))
        # display_image(filtered_image, im_path)
        gtm_image = imread(gtm_path, as_gray=True)
        # display_image(gtm_image, gtm_path)
        # print(is_8bit_image(gtm_image))
        # print(is_8bit_image(filtered_image))
        # Generate ground truth masks using ImageJ thresholds
        for method, threshold in thresholds_imagej.items():
            threshold_normalized = threshold / 255.0  # Normalize threshold to match the image range
            # Creates a binary mask image of 8 bits
            # binary_mask = create_8_bit_image(filtered_image > threshold_normalized)
            try:
                binary_mask = create_8_bit_image(apply_thresholding_method(filtered_image, method, threshold_normalized))
                # display_image(binary_mask, f"Ground Truth Mask ({method}): Threshold = {threshold}")
                # print(is_8bit_image(binary_mask))
                # print(f"method: {method}     accuracy: {calculate_accuracy(gtm_image, binary_mask)}")
                accuracy = calculate_accuracy(gtm_image, binary_mask)
                # If the binary mask doesn't contains white pixels at all
                if not np.sum(binary_mask):
                    lame_methods.add(method)
                    accuracy = 0
                accuracies[method].append(accuracy)
            except ValueError as e:
                errors.append(e)
        # print(accuracies)
        # print(errors)
print_scores(accuracies)
if lame_methods:
    print("\nLame Methods:")
    for method in lame_methods:
        print(f"The method {method} is lame")

The method Moments is lame
1. Method: Yen | Accuracy over 89 images: 0.9949478299430247
2. Method: Li | Accuracy over 89 images: 0.9927712986978252
3. Method: Triangle | Accuracy over 89 images: 0.992587100254016
4. Method: IsoData | Accuracy over 89 images: 0.9922508175453443
5. Method: Otsu | Accuracy over 89 images: 0.9922164745545119
6. Method: Intermodes | Accuracy over 89 images: 0.9921816172224752
7. Method: Mean | Accuracy over 89 images: 0.9914533422234353
8. Method: Minimum | Accuracy over 89 images: 0.9897455365470286
9. Method: Default | Accuracy over 89 images: 0.9892725462324163
10. Method: MaxEntropy | Accuracy over 89 images: 0.9892725462324163
11. Method: Percentile | Accuracy over 89 images: 0.9892725462324163
12. Method: RenyiEntropy | Accuracy over 89 images: 0.9892725462324163
13. Method: Shanbhag | Accuracy over 89 images: 0.9892725462324163
14. Method: Huang | Accuracy over 89 images: 0.9849225590738018
15. Method: Moments | Accuracy over 89 images: 0.34514354320